In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t
from datetime import timedelta, timezone
import datetime
from pytz import timezone

#localtz = timezone('Europe/London')
#today = datetime.datetime.now()
#today =  localtz.localize(today)
start_time = datetime.datetime.now()- timedelta(days =720)
start_time = localtz.localize(start_time)

data = yf.download(tickers="BTC-USD", start = str(start_time.date()), end=str(today.date()), interval = "1h")
data = pd.DataFrame(data["Close"])



def ema(values, period):
    return values.ewm(span=period).mean()

def fisher_transform(df, period):
    fisher_liste = []
    fisher_liste_ma = []
    for i in range(0, len(df)):
        if i< period:
            fisher_liste.append(np.nan)
        else:
            data_list = df.iloc[i-period : i, :  ]["Close"]
            minus1= min(data_list)
            max1 = max(data_list)
            value = df.iloc[i , :  ]["Close"]
            r = 2 - 2*(max1-value)/(max1-minus1)
            fish = np.log((1+r)/(1-r))*0.5
            fisher_liste.append(fish) 
            fisher_ma = pd.Series(fisher_liste)
            fisher_liste_ma.append(fisher_ma.rolling(window = period).mean())
    return [fisher_liste, fisher_liste_ma]
fisher = (fisher_transform(data, 9))
data["Fisher Transform"] = fisher[0]
data["Fisher Transform MA"] = fisher[1]
#tickers = ['AAPL', 'MSFT', '^GSPC']
##  time period and interval
#yf_period   = "5y"   # 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
#yf_interval = "1d"    # 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3m
#symbols     = ['BTC-USD']



[*********************100%***********************]  1 of 1 completed


<ipython-input-92-aa718cf38794>:36: RuntimeWarning: invalid value encountered in log
  fish = np.log((1+r)/(1-r))*0.5


In [90]:
print(data)



                                  Close  Fisher Transform
2019-08-19 22:00:00+01:00  10719.879883               NaN
2019-08-19 23:00:00+01:00  10779.889648               NaN
2019-08-20 00:00:00+01:00  10918.209961               NaN
2019-08-20 01:00:00+01:00  10905.990234               NaN
2019-08-20 02:00:00+01:00  10888.030273               NaN
...                                 ...               ...
2021-08-08 17:00:00+01:00  43792.625000         -0.247690
2021-08-08 18:00:00+01:00  43725.445312         -0.103502
2021-08-08 19:00:00+01:00  43554.394531         -0.250567
2021-08-08 20:00:00+01:00  43645.937500          0.119169
2021-08-08 21:00:00+01:00  43839.343750          0.370941

[17221 rows x 2 columns]


In [5]:
def trade(df, x, y, start_time, end_time, mov_avg_type, long, short, show_on_graph): 
    #   (df, int,int, datetime, datetime, 
    # MOV_AVG_TYPE : int (1- (SMA-SMA), 2- (EMA-EMA), 3- (EMA-SMA))
    #long, short, show_on_graph : BOOLEAN
    if mov_avg_type == 1:
        index1 = "SMA" + str(x)
        index2 = "SMA" + str(y)
        df[index1] = df["Close"].rolling(window = x).mean()
        df[index2] = df["Close"].rolling(window = y).mean()  
        
    if mov_avg_type == 2:
        index1 = "EMA" + str(x)
        index2 = "EMA" + str(y)
        df[index1] = ema(df["Close"], x)
        df[index2] = ema(df["Close"], y)
        
    if mov_avg_type == 3:
        index1 = "EMA" + str(x)
        index2 = "SMA" + str(y)    
        df[index1] = ema(df["Close"], x)
        df[index2] = df["Close"].rolling(window = y).mean()    
        
    if index1 == index2 :
        return {"Hodling Wallet": 0,"Wallet":0, "Total Return": 0, "Buy Points": 0, "Sell Points" : 0, "Long Trades": 0, "Short Trades":0  , "Win Probability" : 0 ,str(index1):None,  str(index2):None , "No of Trade" : 0}

    cross_df = pd.DataFrame()
    cross_df[index1] = df.loc[start_time.date():end_time.date(), index1]
    cross_df[index2] = df.loc[start_time.date():end_time.date(), index2]
    cross_df["Close"] = df.loc[start_time.date():end_time.date(), "Close"]
    
    wallet = 10000
    buy_points = []
    sell_points = []
    cond = 0
    first_changer = 0
    

    for i in range(0,len(cross_df)):
        if cross_df[index1][i]-cross_df[index2][i] >(cross_df[index2][i]/500) and cond != 1:  
            if first_changer == 0:
                first_changer = 1
            cond = 1
            buy_points.append(cross_df["Close"][i])
            sell_points.append(np.nan)
        elif (cross_df[index1][i] - cross_df[index2][i]) < -cross_df[index2][i]/500 and cond != -1:
            if first_changer == 0:
                first_changer = -1
            cond = -1
            sell_points.append(cross_df["Close"][i])
            buy_points.append(np.nan)
        else:
            buy_points.append(np.nan)
            sell_points.append(np.nan)

    sell_points = pd.Series(sell_points).fillna(0)
    buy_points= pd.Series(buy_points).fillna(0)
    
    geniune_sell_points = []
    geniune_buy_points = []
    
    for every in sell_points:
        if every != 0:
            geniune_sell_points.append(every)
    for every in buy_points:
        if every != 0:
            geniune_buy_points.append(every)
    
    
    if first_changer == 1:

        if len(geniune_buy_points) > len(geniune_sell_points):
            geniune_sell_points.append(cross_df["Close"][-1])
            
        for k in range(0,len(geniune_sell_points)-1):
            if long == True:
                wallet += wallet*(geniune_sell_points[k]-geniune_buy_points[k])/geniune_buy_points[k]
            if short == True:
                wallet += wallet*(geniune_sell_points[k]-geniune_buy_points[k+1])/geniune_sell_points[k]
        if long==True:
            wallet += wallet*(geniune_sell_points[-1]-geniune_buy_points[-1])/geniune_buy_points[-1]
            

    elif first_changer == -1:

        if len(geniune_buy_points) == len(geniune_sell_points) :
            geniune_sell_points.append(cross_df["Close"][-1])
            for k in range(0,len(geniune_buy_points)):
                if short == True:
                    wallet += wallet*(geniune_sell_points[k]-geniune_buy_points[k])/geniune_sell_points[k]
                if long == True:
                    wallet += wallet*(geniune_sell_points[k+1]-geniune_buy_points[k])/geniune_buy_points[k]

        elif len(geniune_sell_points) > len(geniune_buy_points):
            geniune_buy_points.append(cross_df["Close"][-1])
            for k in range(0,len(geniune_sell_points)-1):
                if short == True:
                    wallet += wallet*(geniune_sell_points[k]-geniune_buy_points[k])/geniune_sell_points[k]
                if long == True:
                    wallet +=  wallet*((geniune_sell_points[k+1]-geniune_buy_points[k])/geniune_buy_points[k])
            if short == True:
                wallet += wallet*(geniune_sell_points[-1]-geniune_buy_points[-1])/geniune_sell_points[-1]

    buy_points = buy_points.replace(0, np.nan)
    sell_points = sell_points.replace(0, np.nan)

    wins =0
    losses= 0

    if len(geniune_buy_points) == len(geniune_sell_points) and first_changer == 1:
        trade_open_close_long = [[geniune_buy_points[i], geniune_sell_points[i] ] for i in range(0,len(geniune_buy_points)) ]
        trade_open_close_short = [[geniune_sell_points[i], geniune_buy_points[i+1] ] for i in range(0,len(geniune_buy_points)-1)]
        if long == True:
            for every in trade_open_close_long:
                if every[0] < every[1]:
                    wins += 1
                elif every[0] > every[1]:
                    losses += 1
        if short == True:
            for every in trade_open_close_short:
                if every[0] > every[1]:
                    wins += 1
                elif every[0] < every[1]:
                    losses += 1       
            
    elif len(geniune_buy_points) == len(geniune_sell_points) and first_changer == -1:
        trade_open_close_long = [[geniune_buy_points[i], geniune_sell_points[i+1] ] for i in range(0,len(geniune_buy_points)-1) ]
        trade_open_close_short = [[geniune_sell_points[i], geniune_buy_points[i] ] for i in range(0,len(geniune_buy_points)) ]
        if long == True:
            for every in trade_open_close_long:
                if every[0] < every[1]:
                    wins += 1
                elif every[0] > every[1]:
                    losses += 1
        if short == True:
            for every in trade_open_close_short:
                if every[0] > every[1]:
                    wins += 1
                elif every[0] < every[1]:
                    losses += 1    
    elif len(geniune_sell_points) > len(geniune_buy_points) and first_changer == -1:
        trade_open_close_long = [[geniune_buy_points[i], geniune_sell_points[i+1] ] for i in range(0,len(geniune_buy_points)) ]
        trade_open_close_short = [[geniune_sell_points[i], geniune_buy_points[i] ] for i in range(0,len(geniune_sell_points)-1) ]
        if long == True:
            for every in trade_open_close_long:
                if every[0] < every[1]:
                    wins += 1
                elif every[0] > every[1]:
                    losses += 1
        if short == True:
            for every in trade_open_close_short:
                if every[0] > every[1]:
                    wins += 1
                elif every[0] < every[1]:
                    losses += 1        
            
    
    
    win_prob = wins/(wins+losses)
    
    total_return = ((wallet)*100/10000) - 100
    hodling_wallet = 10000*(cross_df["Close"][-1])/cross_df["Close"][0] 
    
    if long == True and short == True:
        num_of_trade = len(trade_open_close_long)+len(trade_open_close_short)
    elif long == True:
        num_of_trade = len(trade_open_close_long)
    elif short == True:
        num_of_trade = len(trade_open_close_short)
        
    if show_on_graph == True:
        plt.style.use('seaborn-bright')
        plt.figure(figsize= (120,5))
        plt.plot(cross_df["Close"], linewidth = 3, alpha = 0.2)
        plt.plot(cross_df[index1], linewidth=1, alpha = 0.5)
        plt.plot(cross_df[index2], linewidth=2, alpha = 0.5)
        plt.scatter(cross_df.index, buy_points, marker = '2', color = 'green')
        plt.scatter(cross_df.index, sell_points, marker = '1', color = 'red')
    return {"Wallet":wallet, "Hodling Wallet": hodling_wallet, "Total Return": total_return, "Long Trades": trade_open_close_long, "Short Trades": trade_open_close_short  , "Win Probability" : win_prob , "No of Trade" : num_of_trade}




In [9]:
trade( data, 20, 20, start_time, start_time+timedelta(days =719, hours = 23), 3, True,True, False)

{'Wallet': 141523.83387277168,
 'Hodling Wallet': 40808.70485240621,
 'Total Return': 1315.2383387277168,
 'Long Trades': [[10158.08984375, 10327.01953125],
  [10137.4501953125, 10310.3603515625],
  [9466.23046875, 10397.8603515625],
  [10744.0595703125, 10387.6904296875],
  [10429.3896484375, 10434.91015625],
  [10353.240234375, 10066.48046875],
  [10031.1201171875, 10320.490234375],
  [10226.4296875, 9824.1904296875],
  [10056.7197265625, 10180.5595703125],
  [10009.759765625, 9694.599609375],
  [8432.1103515625, 7941.60009765625],
  [8058.0498046875, 8101.8798828125],
  [8142.77978515625, 8353.2099609375],
  [8391.650390625, 8134.06982421875],
  [8198.2099609375, 7945.2099609375],
  [8006.5400390625, 8266.58984375],
  [8460.7001953125, 8632.98046875],
  [8326.91015625, 8293.5498046875],
  [8024.4599609375, 7966.59375],
  [7973.8037109375, 8257.12890625],
  [7474.998046875, 9250.9560546875],
  [9337.1650390625, 9456.80859375],
  [9461.23828125, 9136.611328125],
  [9205.7265625, 9156.

In [12]:

def conf_interval( trade_time_interval, x, y, mov_avg_type, long, short, confidentiality):    #trade_type: cross, sma - sma, ema-ema  (x,y): time ranges total_time: 10y, 1y vs.
    
    

    time_ranges = []
    end_time = start_time +datetime.timedelta(days =trade_time_interval) 
    time_ranges.append([start_time, end_time])
    interval = trade_time_interval
    
    while interval < 720:
        end_time = time_ranges[-1][-1] +datetime.timedelta(days =trade_time_interval) 
        time_ranges.append([time_ranges[-1][-1], end_time ])
        interval += trade_time_interval
    time_ranges[-1][1] = datetime.datetime.now()- timedelta(days =1)
    
    returns = []
    hodl_wallets = []
    win_probs = []
    
    highest_trade = {"Trade time": 0, "Trade Return": 0}
    lowest_trade = {"Trade time": 0, "Trade Return": 0}
 
    for every in time_ranges:
        
        

        start = every[0]
        end = every[1]
        
        trade1 = trade(data, x, y, start, end, mov_avg_type, long, short, False )

                
        trade_return = (trade1["Total Return"])
        returns.append(trade_return)
                            
        win_probs.append(trade1["Win Probability"]) 
    
        if trade_return > highest_trade["Trade Return"]:
                            
            highest_trade["Trade Return"] = trade_return
            highest_trade["Trade time"] = [str(i.date()) for i in every]
        
        elif trade_return < lowest_trade["Trade Return"]:
                            
            lowest_trade["Trade Return"] = trade_return
            lowest_trade["Trade time"] = [str(i.date()) for i in every]
        
        hodl_wallets.append(trade1["Hodling Wallet"])
        
          
                            

    mean_return = sum(returns) / len(returns)
    mean_win_prob = sum(win_probs) / len(returns)
    
    std_of_mean_return = (sum([((x - mean_return) ** 2) for x in returns]) / len(returns))**(0.5)
    std_of_mean_win_prob = (sum([((x - mean_win_prob) ** 2) for x in win_probs]) / len(returns))**(0.5)
    
    half_length1 = ((t.ppf( confidentiality + ((1-confidentiality)/2), len(returns) - 1 ))*std_of_mean_return)/((len(returns))**(0.5))
    CI_of_Mean_Return = (mean_return - half_length1 , mean_return + half_length1)
    
    half_length2 = ((t.ppf( confidentiality + ((1-confidentiality)/2), len(returns) - 1 ))*std_of_mean_win_prob)/((len(returns))**(0.5))
    CI_of_Mean_Win_Prob = (mean_win_prob - half_length2, mean_win_prob + half_length2 )
    
    mean_hodl_wallet = sum(hodl_wallets) / len(hodl_wallets)
    
    return {"Mean Return": (mean_return),  "CI of Mean of Return": CI_of_Mean_Return,
            "Mean Win Probability": mean_win_prob, "CI of Win Probability": CI_of_Mean_Win_Prob,
             "Highest Trade" : highest_trade, "Lowest Trade": lowest_trade,
            "Mean Hodl Wallet":  mean_hodl_wallet }
                            

In [27]:

def find_best_pair(trade_time_interval, mov_avg_type, long, short, confidentiality):
    if mov_avg_type == 1:
        index1 = "SMA" 
        index2 = "SMA" 
    if mov_avg_type == 2:
        index1 = "EMA" 
        index2 = "EMA" 
    if mov_avg_type == 3:
        index1 = "EMA" 
        index2 = "SMA"   
    
    pair_type = str(index1 + " - " + index2)
    highest_mean_return = {"return": 0, pair_type : []}
    highest_mean_win_prob = {"Win Probability": 0, pair_type : []}
    for i in range (18,22,2):
        for j in range (15,25,5):
            if conf_interval(30, i, j, mov_avg_type, long, short, confidentiality = 0.7)["Mean Return"] > highest_mean_return["return"]:
                highest_mean_return["return"] =  conf_interval(30, i, j, mov_avg_type, long, short, confidentiality = 0.7)["Mean Return"]
                if len(highest_mean_return[pair_type])<2:

                    highest_mean_return[pair_type].append(i)
                    highest_mean_return[pair_type].append(j)

                elif len(highest_mean_return[pair_type])==2:
                    highest_mean_return[pair_type][0] = i
                    highest_mean_return[pair_type][1] = j

            if conf_interval(30, i, j, mov_avg_type, long, short, confidentiality = 0.7)["Mean Win Probability"] > highest_mean_win_prob["Win Probability"]:
                highest_mean_win_prob["Win Probability"] =  conf_interval(30, i, j, mov_avg_type, long, short, confidentiality = 0.7)["Mean Win Probability"]
                if len(highest_mean_win_prob[pair_type])<2:

                    highest_mean_win_prob[pair_type].append(i)
                    highest_mean_win_prob[pair_type].append(j)

                elif len(highest_mean_win_prob[pair_type])==2:
                    highest_mean_win_prob[pair_type][0] = i
                    highest_mean_win_prob[pair_type][1] = j
    return [highest_mean_return.values(), highest_mean_win_prob.values()]

In [35]:
print(find_best_pair(30,3, True, True, 0.7))

[dict_values([14.789475675602963, [20, 20]]), dict_values([0.5784871615991122, [20, 15]])]


In [31]:
trade( data, 20, 20, start_time, start_time+timedelta(days =719, hours = 23), 1, True,True, False)

{'Hodling Wallet': 0,
 'Wallet': 0,
 'Total Return': 0,
 'Buy Points': 0,
 'Sell Points': 0,
 'Long Trades': 0,
 'Short Trades': 0,
 'Win Probability': 0,
 'SMA20': None,
 'No of Trade': 0}

20.36158495870749

In [ ]:
"""def trade_cross_long_short(price_df, x, y):
    cross_df = pd.DataFrame()
    index1 = "EMA" + str(min(x,y))
    index2 = "SMA" + str(max(x,y))
    cross_df[index1] = ema(price_df["Close"], min(x,y))
    cross_df[index2] = price_df["Close"].rolling(window = max(x,y)).mean()
    
    
    wallet = 10000
    buy_points = []
    sell_points = []
    cond = 0
    first_changer = 0

    for i in range(0,len(price_df)):
        if cross_df[index1][i]-cross_df[index2][i] > (cross_df[index2][i]/500) and cond != 1: #cross_df[index1][i]>cross_df[index2][i]
            if first_changer == 0:
                first_changer = 1
            cond = 1
            buy_points.append(price_df["Close"][i])
            sell_points.append(np.nan)
        elif (cross_df[index1][i] - cross_df[index2][i]) < -cross_df[index2][i]/500 and cond != -1:#cross_df[index1][i] < cross_df[index2][i] and cond != -1:
            if first_changer == 0:
                first_changer = -1
            cond = -1
            sell_points.append(price_df["Close"][i])
            buy_points.append(np.nan)
        else:
            buy_points.append(np.nan)
            sell_points.append(np.nan)
            
            


    sell_points = pd.Series(sell_points).fillna(0)
    buy_points= pd.Series(buy_points).fillna(0)
    
    geniune_sell_points = []
    geniune_buy_points = []
    
    for every in sell_points:
        if every != 0:
            geniune_sell_points.append(every)
    for every in buy_points:
        if every != 0:
            geniune_buy_points.append(every)
    
    if first_changer == 1:
        #for k in range(0,len(sell_points)):
         #   wallet*= (sell_points[k] - buy_points[k])/buy_points[k]
        if len(geniune_buy_points) > len(geniune_sell_points):
            geniune_sell_points.append(price_df["Close"][-1])
        for k in range(0,len(geniune_sell_points)-1):
            wallet += wallet*(geniune_sell_points[k]-geniune_buy_points[k])/geniune_buy_points[k]
            wallet += wallet*(geniune_sell_points[k]-geniune_buy_points[k+1])/geniune_sell_points[k]
        wallet += wallet*(geniune_sell_points[-1]-geniune_buy_points[-1])/geniune_buy_points[-1]
            

    elif first_changer == -1:

        if len(geniune_buy_points) == len(geniune_sell_points) :
            geniune_sell_points.append(price_df["Close"][-1])
            for k in range(0,len(geniune_buy_points)):
                wallet += wallet*(geniune_sell_points[k]-geniune_buy_points[k])/geniune_sell_points[k]
                wallet += wallet*(geniune_sell_points[k+1]-geniune_buy_points[k])/geniune_buy_points[k]

        elif len(geniune_sell_points) > len(geniune_buy_points):
            geniune_buy_points.append(price_df["Close"][-1])
            for k in range(0,len(geniune_sell_points)-1):
                wallet += wallet*(geniune_sell_points[k]-geniune_buy_points[k])/geniune_sell_points[k]
                wallet +=  wallet*((geniune_sell_points[k+1]-geniune_buy_points[k])/geniune_buy_points[k])
            wallet += wallet*(geniune_sell_points[-1]-geniune_buy_points[-1])/geniune_sell_points[-1]

    buy_points = buy_points.replace(0, np.nan)
    sell_points = sell_points.replace(0, np.nan)

    long_wins =0
    long_losses= 0
    short_wins = 0
    short_losses = 0
    if len(geniune_buy_points) == len(geniune_sell_points) and first_changer == 1:
        trade_open_close_long = [[geniune_buy_points[i], geniune_sell_points[i] ] for i in range(0,len(geniune_buy_points)) ]
        trade_open_close_short = [[geniune_sell_points[i], geniune_buy_points[i+1] ] for i in range(0,len(geniune_buy_points)-1)]
        for every in trade_open_close_long:
            if every[0] < every[1]:
                long_wins += 1
            elif every[0] > every[1]:
                long_losses += 1
        for every in trade_open_close_short:
            if every[0] > every[1]:
                short_wins += 1
            elif every[0] < every[1]:
                short_losses += 1       
            
    elif len(geniune_buy_points) == len(geniune_sell_points) and first_changer == -1:
        trade_open_close_long = [[geniune_buy_points[i], geniune_sell_points[i+1] ] for i in range(0,len(geniune_buy_points)-1) ]
        trade_open_close_short = [[geniune_sell_points[i], geniune_buy_points[i] ] for i in range(0,len(geniune_buy_points)) ]
        for every in trade_open_close_long:
            if every[0] < every[1]:
                long_wins += 1
            elif every[0] > every[1]:
                long_losses += 1
        for every in trade_open_close_short:
            if every[0] > every[1]:
                short_wins += 1
            elif every[0] < every[1]:
                short_losses += 1       
    elif len(geniune_sell_points) > len(geniune_buy_points) and first_changer == -1:
        trade_open_close_long = [[geniune_buy_points[i], geniune_sell_points[i+1] ] for i in range(0,len(geniune_buy_points)) ]
        trade_open_close_short = [[geniune_sell_points[i], geniune_buy_points[i] ] for i in range(0,len(geniune_sell_points)-1) ]
        for every in trade_open_close_long:
            if every[0] < every[1]:
                long_wins += 1
            elif every[0] > every[1]:
                long_losses += 1
        for every in trade_open_close_short:
            if every[0] > every[1]:
                short_wins += 1
            elif every[0] < every[1]:
                short_losses += 1       
    
    
    win_prob_long = long_wins/(long_wins+long_losses)
    win_prob_short = short_wins/(short_wins+short_losses)
    
    total_return = ((wallet)*100/10000) - 100
    return {"Wallet":wallet, "Total Return": total_return, "Buy Points": buy_points, "Sell Points" : sell_points, 
            "Long Trades": trade_open_close_long, "Short Trades": trade_open_close_short  ,
            "Win Probability Long" : win_prob_long , "Win Probability Short" : win_prob_short ,
            str(index1):cross_df[index1],  str(index2):cross_df[index2] , "No of Trade" : len(trade_open_close_long)+len(trade_open_close_short)}



"""
                